Lib

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import nltk

from textblob import TextBlob
from googletrans import Translator

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pyvi import ViTokenizer, ViUtils
from vncorenlp import VnCoreNLP


in ra 5 mẫu dl

In [2]:
data = pd.read_csv("sentiments_v2_1_1000.csv")
comments = data['comment'].tolist()
data.head()

,userid,mtime,rating_star,comment
0,69545822,20/05/2021 11:29,5,"Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê..."
1,49021104,08/05/2021 22:10,5,Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...
2,275113953,30/06/2021 21:42,5,Sản phẩm giống trên hình. Son khá lì. Thời gia...
3,470905623,30/06/2021 21:42,5,Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...
4,321034368,19/05/2021 12:08,5,Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...


chuẩn hóa và loại bỏ trùng lắp dl (*Case: DAU TAY TRANG, có thể hiểu 2 nghĩa trong tiếng Việt 'Dầu tẩy trang' / 'Dâu tây trắng' => Làm sao để phân biệt được?)

In [3]:
import unicodedata

unique_comments = list(set(comments))

def remove_vietnamese_accents(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

normalized_comments = list(set([remove_vietnamese_accents(comment).lower() for comment in comments]))
print(len(normalized_comments), "là số lượng bình luận sau khi chuẩn hóa và loại bỏ trùng lặp.")


999 là số lượng bình luận sau khi chuẩn hóa và loại bỏ trùng lặp.


VADER Sentiment Analysis + googletrans

In [9]:
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

# nhập từ bàn phím
translator = Translator()
analyzer = SentimentIntensityAnalyzer()

# Hàm phân tích cảm xúc cho câu tiếng Việt và trả về cả nhãn và giá trị compound
def classify_sentiment_vietnamese(comment):
    # Dịch câu sang tiếng Anh
    translated_text = translator.translate(comment, src='vi', dest='en').text
    # Phân tích cảm xúc của câu đã dịch
    sentiment_score = analyzer.polarity_scores(translated_text)
    compound_score = sentiment_score['compound'] #Compound này lấy từ cái gần tọa độ 
    
    # Phân loại cảm xúc dựa trên giá trị compound
    if compound_score > 0:
        sentiment = 'Tích cực'
    elif compound_score < 0:
        sentiment = 'Tiêu cực'
    else:
        sentiment = 'Trung lập'
    
    # Trả về nhãn cảm xúc và giá trị compound
    return sentiment, compound_score

# Ví dụ câu tiếng Việt
comment = "Không có có"
sentiment, compound_score = classify_sentiment_vietnamese(comment)
print(f"Cảm xúc: {sentiment}, Compound Score: {compound_score}")


Cảm xúc: Trung lập, Compound Score: 0.0


In [4]:
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

translator = Translator()
analyzer = SentimentIntensityAnalyzer()

# Hàm phân tích cảm xúc cho câu tiếng Việt và trả về cả nhãn và giá trị compound
def classify_sentiment_vietnamese(comment):
    # Dịch câu sang tiếng Anh
    translated_text = translator.translate(comment, src='vi', dest='en').text  # Sử dụng .text để lấy văn bản đã dịch
    # Phân tích cảm xúc của câu đã dịch
    sentiment_score = analyzer.polarity_scores(translated_text)
    compound_score = sentiment_score['compound']
    
    # Phân loại cảm xúc dựa trên giá trị compound
    if compound_score > 0:
        sentiment = 'Tích cực'
    elif compound_score < 0:
        sentiment = 'Tiêu cực'
    else:
        sentiment = 'Trung lập'
    
    return sentiment, compound_score

# Sử dụng hàm để phân tích cảm xúc cho các bình luận
results = [classify_sentiment_vietnamese(comment) for comment in comments]

# Chuyển kết quả phân tích thành DataFrame
sentiments_df = pd.DataFrame(results, columns=['Sentiment', 'Compound Score'])

# Thêm các kết quả vào dữ liệu gốc
data['Sentiment'] = sentiments_df['Sentiment']
data['Compound Score'] = sentiments_df['Compound Score']

# In ra 10 dòng đầu tiên với kết quả phân tích
print(data.head(10))

      userid             mtime  rating_star  \
0   69545822  20/05/2021 11:29            5   
1   49021104  08/05/2021 22:10            5   
2  275113953  30/06/2021 21:42            5   
3  470905623  30/06/2021 21:42            5   
4  321034368  19/05/2021 12:08            5   
5   23364385  26/05/2021 15:04            5   
6  124689928  05/06/2021 13:52            5   
7   70572173  30/06/2021 12:58            5   
8   58074290  14/05/2021 16:57            5   
9  354970721  16/05/2021 10:21            5   

                                             comment  Sentiment  \
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   Tích cực   
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...   Tích cực   
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  Trung lập   
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...   Tích cực   
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  Trung lập   
5  Xinh lắm các bạn ơi bling bling lắm giao hàng ...   Tích cực   
6  shop giao 

VnCoreNLP + VADER + TextBlob 

In [5]:
from vncorenlp import VnCoreNLP

from textblob import TextBlob

VnCoreNLP_PATH = "E:/nam4/hk1/doan/VnCoreNLP/VnCoreNLP-1.1.1.jar"
annotator = VnCoreNLP(VnCoreNLP_PATH, annotators="wseg,pos", max_heap_size='-Xmx2g')

# Khởi tạo VADER và Translator
analyzer = SentimentIntensityAnalyzer()
translator = Translator()

# Hàm dịch văn bản từ tiếng Việt sang tiếng Anh
def translate_to_english(text):
    try:
        return translator.translate(text, src='vi', dest='en').text
    except Exception as e:
        return text  # Nếu lỗi, trả lại văn bản gốc

# Hàm phân tích cảm xúc sử dụng VADER và TextBlob
def analyze_sentiment_vietnamese(comment):
    # Bước 1: Dịch văn bản từ tiếng Việt sang tiếng Anh
    translated_comment = translate_to_english(comment)
    
    # Bước 2: Sử dụng VADER để phân tích cảm xúc
    sentiment_score_vader = analyzer.polarity_scores(translated_comment)
    sentiment_score_blob = TextBlob(translated_comment).sentiment.polarity

    # Bước 3: Phân tích cảm xúc từ VADER
    sentiment_vader = sentiment_score_vader['compound']
    
    # Quyết định cảm xúc từ VADER
    if sentiment_vader > 0.1:
        sentiment = "Tích cực"
        score = sentiment_vader
    elif sentiment_vader < -0.1:
        sentiment = "Tiêu cực"
        score = sentiment_vader
    else:
        sentiment = "Trung lập"
        score = 0
    
    return sentiment, score

# Phân tích cảm xúc cho tất cả các bình luận
results = [analyze_sentiment_vietnamese(comment) for comment in comments]

# Chuyển kết quả phân tích thành DataFrame
sentiments_df = pd.DataFrame(results, columns=['Sentiment', 'Compound Score'])

# Thêm các kết quả vào dữ liệu gốc để xem bình luận kèm nhãn cảm xúc
data['Sentiment'] = sentiments_df['Sentiment']
data['Compound Score'] = sentiments_df['Compound Score']

# In ra 10 dòng đầu tiên với kết quả phân tích
print(data.head(10))

      userid             mtime  rating_star  \
0   69545822  20/05/2021 11:29            5   
1   49021104  08/05/2021 22:10            5   
2  275113953  30/06/2021 21:42            5   
3  470905623  30/06/2021 21:42            5   
4  321034368  19/05/2021 12:08            5   
5   23364385  26/05/2021 15:04            5   
6  124689928  05/06/2021 13:52            5   
7   70572173  30/06/2021 12:58            5   
8   58074290  14/05/2021 16:57            5   
9  354970721  16/05/2021 10:21            5   

                                             comment  Sentiment  \
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   Tích cực   
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...   Tích cực   
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  Trung lập   
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...   Tích cực   
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  Trung lập   
5  Xinh lắm các bạn ơi bling bling lắm giao hàng ...   Tích cực   
6  shop giao 

BERT (PhoBERT)

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load tokenizer và model PhoBERT cho tiếng Việt
tokenizer = BertTokenizer.from_pretrained('vinai/phobert-base')
model = TFBertForSequenceClassification.from_pretrained('vinai/phobert-base')

# Tiền xử lý văn bản tiếng Việt
text = "hôm nay xe của người tôi ghét bị ngập nước "
inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=128)

# Dự đoán cảm xúc
outputs = model(inputs)
logits = outputs.logits
predicted_class = tf.argmax(logits, axis=-1).numpy()

print("Predicted class:", predicted_class)

if predicted_class == 1:
    print("tích cực")
else:
    print ("tiêu cực")

In [3]:
from transformers import pipeline

# Khởi tạo mô hình phân tích cảm xúc sử dụng XLM-R (mô hình BERT đa ngôn ngữ)
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Hàm phân tích cảm xúc cho câu tiếng Việt mà không cần liệt kê từ khóa
def classify_sentiment_vietnamese(comment):
    # Sử dụng mô hình đã huấn luyện để phân tích cảm xúc
    result = classifier(comment)
    
    # Trả về nhãn cảm xúc và giá trị cảm xúc
    label = result[0]['label']
    score = result[0]['score']
    
    # Phân loại cảm xúc theo nhãn của mô hình
    if label == '5 stars':
        sentiment = 'Tích cực'
        compound_score = 1
    elif label == '1 star':
        sentiment = 'Tiêu cực'
        compound_score = -1
    else:
        sentiment = 'Trung lập'
        compound_score = 0
    
    return sentiment, compound_score

# Đọc dữ liệu từ file CSV
data = pd.read_csv("sentiments_v2_1_1000.csv")

# Lấy danh sách các bình luận
comments = data['comment'].tolist()

# Phân tích cảm xúc cho tất cả các bình luận
results = [classify_sentiment_vietnamese(comment) for comment in comments]

# Chuyển kết quả phân tích thành DataFrame
sentiments_df = pd.DataFrame(results, columns=['Sentiment', 'Compound Score'])

# Thêm các kết quả vào dữ liệu gốc để xem bình luận kèm nhãn cảm xúc
data['Sentiment'] = sentiments_df['Sentiment']
data['Compound Score'] = sentiments_df['Compound Score']

# In ra 10 dòng đầu tiên với kết quả phân tích
print(data.head(10))


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


      userid             mtime  rating_star  \
0   69545822  20/05/2021 11:29            5   
1   49021104  08/05/2021 22:10            5   
2  275113953  30/06/2021 21:42            5   
3  470905623  30/06/2021 21:42            5   
4  321034368  19/05/2021 12:08            5   
5   23364385  26/05/2021 15:04            5   
6  124689928  05/06/2021 13:52            5   
7   70572173  30/06/2021 12:58            5   
8   58074290  14/05/2021 16:57            5   
9  354970721  16/05/2021 10:21            5   

                                             comment  Sentiment  \
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   Tích cực   
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...  Trung lập   
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  Trung lập   
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...  Trung lập   
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  Trung lập   
5  Xinh lắm các bạn ơi bling bling lắm giao hàng ...   Tích cực   
6  shop giao 

Tensorflow

In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# In ra tên các cột để chắc chắn rằng cột 'comment' và 'sentiment' tồn tại
print(data.columns)

# Nếu không có cột 'sentiment', tạo nhãn cảm xúc
if 'sentiment' not in data.columns:
    def label_sentiment(comment):
        if 'good' in comment or 'love' in comment:
            return 1  # Tích cực
        else:
            return 0  # Tiêu cực
    data['sentiment'] = data['comment'].apply(label_sentiment)

# 2. Tiền xử lý dữ liệu
max_features = 10000  # Giới hạn số từ để giữ lại
max_len = 200  # Độ dài tối đa của mỗi comment

# Tokenizer để chuyển văn bản thành các chỉ số
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['comment'])  # Tạo từ điển các từ trong dữ liệu (cột 'comment')
x_data = tokenizer.texts_to_sequences(data['comment'])  # Chuyển đổi văn bản thành chỉ số
x_data = pad_sequences(x_data, maxlen=max_len)  # Padding cho các câu có độ dài khác nhau

# 3. Mã hóa nhãn cảm xúc
y_data = data['sentiment'].values  # Nhãn cảm xúc (1 cho tích cực, 0 cho tiêu cực)

# 4. Xây dựng mô hình phân loại cảm xúc
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

# 5. Compile mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Huấn luyện mô hình
model.fit(x_data, y_data, epochs=5, batch_size=64, validation_split=0.2)

# 7. Dự đoán cảm xúc cho tất cả các comment trong file CSV
predictions = model.predict(x_data)

# Chuyển đổi dự đoán thành nhãn (1 là tích cực, 0 là tiêu cực)
predicted_labels = [1 if pred >= 0.5 else 0 for pred in predictions]

# Thêm kết quả dự đoán vào DataFrame
data['predicted_sentiment'] = predicted_labels

# 8. Lưu kết quả vào file CSV mới
data.to_csv('reviews_with_sentiment_predictions.csv', index=False)

# 9. Hiển thị kết quả
print(data[['comment', 'sentiment', 'predicted_sentiment']])


Index(['userid', 'mtime', 'rating_star', 'comment', 'sentiment',
       'predicted_sentiment'],
      dtype='object')


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 497ms/step - accuracy: 0.8966 - loss: 0.5530 - val_accuracy: 1.0000 - val_loss: 0.0204
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 348ms/step - accuracy: 0.9997 - loss: 0.0156 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 317ms/step - accuracy: 0.9994 - loss: 0.0059 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - accuracy: 0.9964 - loss: 0.0258 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 309ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 1.0000 - val_loss: 9.3796e-04
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step
                                               comment  sentiment  \
0    Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...          0   
1    Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...          0   
2    Sản phẩm giống trên hình. Son khá lì. Thời gia...          0   
3    Son xinh xĩu lun á shopp ơii.vỏ son nhì

In [ ]:
# Đọc file CSV
file_path = "sentiments_v2_7001_7230.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print(data.columns)

Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')


Dụng thư viện textblob

In [6]:
# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Chỉ kiểm tra nếu cột 'comment' tồn tại
if 'comment' in data.columns:
    # Khởi tạo công cụ dịch
    translator = Translator()

    # Hàm dịch và phân tích cảm xúc
    def analyze_sentiment(text):
        # Dịch văn bản sang tiếng Anh
        translated_text = translator.translate(text, src='vi', dest='en').text
        # Phân tích cảm xúc bằng TextBlob
        blob = TextBlob(translated_text)
        return blob.sentiment.polarity  # Trả về điểm cảm xúc từ -1 (tiêu cực) đến 1 (tích cực)

    # Áp dụng hàm cho 5 bình luận đầu tiên
    data['sentiment'] = data['comment'].head(5).apply(analyze_sentiment)

    # Hiển thị 5 bình luận đầu tiên với điểm cảm xúc
    print(data[['comment', 'sentiment']].head(5))
else:
    print("Cột 'comment' không tồn tại trong DataFrame.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')


                                             comment  sentiment
0  Đồ giống hình, nhìn khá xinh, giá rẻ, ship siê...   0.295833
1  Giao hàng nhanh. Mình đặt hộ nên ko biết chất ...   0.391667
2  Sản phẩm giống trên hình. Son khá lì. Thời gia...  -0.025000
3  Son xinh xĩu lun á shopp ơii.vỏ son nhìn cute ...   0.850000
4  Cách xoá MỤN ĐẦU ĐEN mũi bằng DẦU TẨY TRANG. K...  -0.125000


In [12]:
# Đọc file CSV và bỏ qua các dòng lỗi
file_path = "sentiments_v2_7001_7230.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Giả sử cột chứa văn bản là 'text' (thay bằng tên cột tương ứng nếu khác)
if 'comment' in data.columns:
    # Tách từ cho từng dòng trong cột 'text' và lưu vào một cột mới 'tokenized_text'
    data['tokenized_text'] = data['comment'].apply(ViTokenizer.tokenize)

    output_file_path = "sentiments_tk_7001_7230.csv"
    data.to_csv(output_file_path, index=False)  # Không lưu chỉ mục (index)
    
    print(data[['comment', 'tokenized_text']].head())
else:
    print("Không tìm thấy cột 'comment' trong DataFrame. Vui lòng kiểm tra lại tên cột.")

Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')
                                             comment  \
0             Giá rẻ. Mua xịt khử mùi ăn uóng khá ổn   
1  thơm lắm mn oi có chai đào là khi xịt hòi là h...   
2  eidighgghuđhuêd Dbddnd ĐĐ D D c chr ý nghĩa là...   
3    Thơm nha các bạn nhưng k giữ đc mùi lâu đâu nhé   
4                         Sản phẩm giống hình xài ok   

                                      tokenized_text  
0            Giá rẻ . Mua xịt khử mùi ăn uóng khá ổn  
1  thơm lắm mn oi có chai đào là khi xịt hòi là h...  
2  eidighgghuđhuêd Dbddnd ĐĐ D D c chr ý_nghĩa_là...  
3    Thơm nha các bạn nhưng k giữ đc mùi lâu đâu nhé  
4                         Sản_phẩm giống hình xài ok  


In [14]:
# Danh sách các stop words tiếng Việt (có thể mở rộng thêm)
stop_words = set([
    "và", "của", "là", "đã", "đang", "nên", "nhưng", "với", "bằng", "theo", 
    "làm", "từ", "trong", "lúc", "mà", "này", "có", "chưa", "vì", "sẽ", "khi",
    "được", "không", "tôi", "bạn", "chúng", "mình", "một", "nhiều", "vài", 
    "cũng", "nào", "nếu", "hay", "kể", "vừa", "nên", "có thể", "làm", "mà"
])

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Giả sử cột chứa văn bản là 'comment' (thay bằng tên cột khác nếu cần)
if 'comment' in data.columns:
    # Tách từ trong cột 'comment'
    data['tokenized_comment'] = data['comment'].apply(ViTokenizer.tokenize)
    
    # Bước loại bỏ stop words
    def remove_stopwords(text):
        words = text.split()
        filtered_words = [word for word in words if word not in stop_words]
        return " ".join(filtered_words)
    
    # Áp dụng loại bỏ stop words
    data['cleaned_comment'] = data['tokenized_comment'].apply(remove_stopwords)

    # Lưu kết quả vào file CSV mới
    output_file_path = "sentiments_v2_7001_7230.csv"
    data.to_csv(output_file_path, index=False)  # Không lưu chỉ mục (index)

    print(f"Dữ liệu đã được lưu vào file: {output_file_path}")
else:
    print("Không tìm thấy cột 'comment' trong DataFrame. Vui lòng kiểm tra lại tên cột.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')
Dữ liệu đã được lưu vào file: sentiments_v2_7001_7230.csv


VnCoreNLP

In [8]:
# Đặt đường dẫn tới mô hình VnCoreNLP
vncorenlp_path = r'D:\Y4 HK1\VnCoreNLP\VnCoreNLP-1.1.1.jar'  # Thay bằng đường dẫn chính xác của bạn

# Khởi tạo VnCoreNLP
vncorenlp = VnCoreNLP(vncorenlp_path)

if 'comment' in data.columns:
    # Áp dụng lemmatization cho mỗi dòng trong cột 'comment'
    def lemmatize_text(text):
    # Dùng VnCoreNLP để annotate văn bản
        annotated = vncorenlp.annotate(text)
        
        # Trích xuất các từ đã được lemmatize từ dữ liệu trả về
        words = []
        for sentence in annotated.get('sentences', []):
            for token in sentence:
                # Kiểm tra nếu 'lemma' tồn tại trong token
                if 'lemma' in token:
                    words.append(token['lemma'])
                else:
                    words.append(token['form'])  # Nếu không có 'lemma', lấy từ gốc
        
        # Trả về các từ đã lemmatized
        return ' '.join(words)

# Áp dụng lemmatization cho mỗi dòng trong cột 'comment'
    data['lemmatized_comment'] = data['comment'].apply(lemmatize_text)

    # Lưu lại kết quả vào file CSV mới
    data.to_csv('lemmatized_sentiments.csv', index=False)  # Lưu vào file mới

    print("Lematization đã hoàn thành và lưu vào file lemmatized_sentiments.csv")
else:
    print("Không tìm thấy cột 'comment' trong DataFrame.")

Lematization đã hoàn thành và lưu vào file lemmatized_sentiments.csv
